# <div style="text-align: center">ECC</div>



<div style="text-align: justify;">
    El cifrado asimétrico es una técnica criptográfica que utiliza pares de claves distintas para encriptar y desencriptar datos. Cada par de claves consiste en una clave pública, que puede ser compartida, y una clave privada, que debe mantenerse en secreto. Esto permite a cualquier persona cifrar datos utilizando la clave pública del destinatario, pero solo el destinatario a través de su clave privada podrá descifrar la información.
    El cifrado con curvas elípticas es una forma de cifrado asimétrico que emplea las curvas elípticas para realizar operaciones criptográficas. Este método ofrece ventajas en términos de eficiencia y seguridad en comparación con otros métodos asimétricos, como RSA, ya que utiliza propiedades de las curvas elípticas para la generación de claves seguras.

El sencillo programa en la parte inferior implementa un sistema básico de cifrado utilizando curvas elípticas (ECC) y una función XOR:

Primero, se genera una clave privada aleatoria a partir de un número aleatorio comprendido entre 0 y el orden de la curva elíptica `secp192r1`. Una vez generada la clave privada, se calcula su correspondiente clave pública. Luego, se definen las funciones para cifrar (enkryptar) y descifrar (desenkryptar) mensajes utilizando XOR y la clave privada correspondiente.

La función `xor_simple(data, clave)` utiliza XOR (or exclusivo) para combinar cada byte del mensaje original con la clave generada dinámicamente. Esto permite que dicho mensaje solo pueda descifrarse utilizando la clave privada.

</div>

<div style="text-align: center;">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*_wRwisFsygokN6YQTIEq2Q.png" alt="Elliptic Curve">
</div>


In [ ]:
pip install tinyec

In [ ]:
import secrets as sc
from tinyec import registry

curva = registry.get_curve('secp192r1')

clavePriv = sc.randbelow(curva.field.n)
clavePub = clavePriv * curva.g
comClavePub = '0' + str(2 + clavePub.y % 2) + str(hex(clavePub.x)[2:])
print("Clave privada: ", hex(clavePriv)[2:])
print("Clave publica: ", comClavePub)

def xor_simple(data, clave):
    return bytes(a ^ b for a, b in zip(data, clave))

def enkryptar(msg, pubClave):
    cifradoClavePriv = sc.randbelow(curva.field.n)
    claveKompartida = cifradoClavePriv * pubClave
    claveKompartidaB = claveKompartida.x.to_bytes((claveKompartida.x.bit_length() + 7) // 8, 'big')
    ciphertxt = xor_simple(msg, claveKompartidaB)
    return ciphertxt, cifradoClavePriv * curva.g

def desenkryptar(cMsg, privClave):
    (cTxt, ciphertxtClavePub) = cMsg
    claveKompartida = privClave * ciphertxtClavePub
    claveKompartidaB = claveKompartida.x.to_bytes((claveKompartida.x.bit_length() + 7) // 8, 'big')
    plaintext = xor_simple(cTxt, claveKompartidaB)
    return plaintext

mensaje = b"Hola, me gustas"
eMsg = enkryptar(mensaje, clavePub)
print("\nMensaje encriptado:", eMsg[0])

deMsg = desenkryptar(eMsg, clavePriv)
print("Mensaje desencriptado:", deMsg.decode('utf-8'))

def main():
    while True:
        plaintext = input("\nIngrese el texto a encriptar (presione Enter para salir): ")
        if not plaintext:
            break

        eMsg = enkryptar(plaintext.encode('utf-8'), clavePub)
        print("Mensaje encriptado: ", eMsg[0])
        deMsg = desenkryptar(eMsg, clavePriv)
        print("Mensaje desencriptado: ", deMsg.decode('utf-8'))


if __name__ == "__main__":
    main()